In [13]:
import numpy as np

In [240]:
def activation(s):
    return 0 if s <= 0 else s

def derivativeActivation(s):
    # return 0 if s <= 0 else 1
    return (1 - tanh(s) ** 2)
    # return (1 - (s) ** 2)

def derivativeActivationW(s):
    # return 0 if s <= 0 else 1
    # return (1 - tanh(s) ** 2)
    return (1 - (s ** 2))


def outputf(s):
    return (1) / ((1) + np.exp(-s))

def derivativeOutput(s):
    return (outputf(s)) * (1 - outputf(s))

def errorf(x_L, y):
    if y == 1:
        return np.log(x_L)
    else:
        return -np.log(1 - x_L)
    
def tanh(s):
    return np.tanh(s)

In [366]:
X = [1, 2]
W = [
    [[0.1,0.2],
     [0.3,0.4]],
    [[0.2],
     [1],
     [-3]],
    [[1],
     [2]]
]

In [20]:
def forwardPropagation(x, weights):
    Xl = np.array(x)
    W = np.array(weights)
    S = []
    X = [x]
    for index,l in enumerate(W):
        wl = np.array(l)
        sl = np.transpose(wl).dot(Xl)
        Xl_before_activation = sl
        if index != len(W) - 1:
            activation_function = np.vectorize(tanh)
            Xl = activation_function(Xl_before_activation)
            Xl = np.insert(Xl, 0, 1, axis=0)
        else:
            output_function = np.vectorize(tanh)
            Xl = output_function(Xl_before_activation)
        X.append(Xl)
        S.append(sl)
    np.delete
    return np.array(X),np.array(S)

In [214]:
x,s = forwardPropagation(X,W)
for i in s:
    print(i)
print('\n',s)

[0.7 1. ]
[-1.48041469]
[-0.80309131]

 [array([0.7, 1. ]) array([-1.48041469]) array([-0.80309131])]


In [367]:
from copy import deepcopy
def backPropagation(X, y_n, s, weights):
    weights_copy = deepcopy(weights)
    g = [None] * len(X)
    X = np.array(X)
    for layer, Xl in enumerate(reversed(X)):
        layer = len(X) - layer - 1
        if layer == len(X) - 1:
            delta = 2 * (Xl[0] - y_n) * derivativeActivation(s[-1][0])
            g[layer] = np.array([delta])
        elif layer > 0:
            derivatives = np.zeros([len(Xl) - 1,len(Xl) - 1])
            for i in range(len(Xl) - 1):
                derivatives[i][i] = derivativeActivationW(Xl[i + 1])
            
            Wl = weights_copy[layer]
            Wl_t = np.array(Wl)
            g[layer] = ((Wl_t).dot((g[layer + 1]).T)[1:]).T.dot(derivatives)
            
            
            print('Wl',Wl)
            print('gl', g[layer + 1])
            print('der', derivatives)
            print('------------------------------')            
            
    g = g[1:]
    
    
    updatedW = weights_copy
    for layer, Xl in enumerate(X[:-1]):
        # Xl = np.array(Xl).reshape(len(Xl),)
        print(Xl)
        print(g[layer])
        print('------------------------------')
        # updatedW[layer] = Xl.T.dot(g[layer])
        updatedW[layer] = np.dot(np.array([Xl]).T, np.array([g[layer]]))
    
    return (updatedW)

In [368]:
g = backPropagation(x, 1, s, W)
for i in g:
    print(i)

Wl [[1], [2]]
gl [-1.85486437]
der [[0.18721543]]
------------------------------
Wl [[0.2], [1], [-3]]
gl [-0.69451848]
der [[0.63473959 0.        ]
 [0.         0.41997434]]
------------------------------
[1, 2]
[-0.44083838  0.87503983]
------------------------------
[1.         0.60436778 0.76159416]
[-0.69451848]
------------------------------
[ 1.         -0.90154565]
[-1.85486437]
------------------------------
[[-0.44083838  0.87503983]
 [-0.88167675  1.75007965]]
[[-0.69451848]
 [-0.41974459]
 [-0.52894122]]
[[-1.85486437]
 [ 1.67224491]]


In [369]:
W

[[[0.1, 0.2], [0.3, 0.4]], [[0.2], [1], [-3]], [[1], [2]]]

In [370]:
def updateWeights(weights, g, alpha):
    nW = deepcopy(weights)
    for i in range(len(nW)):
        for j in range(len(nW[i])):
            nW[i][j] = nW[i][j] - (alpha * g[i][j]) 
    return nW

In [372]:
nW = updateWeights(W, g, 1)
for i in nW:
    print(i)

[array([ 0.54083838, -0.67503983]), array([ 1.18167675, -1.35007965])]
[array([0.89451848]), array([1.41974459]), array([-2.47105878])]
[array([2.85486437]), array([0.32775509])]


In [479]:
# def fit_NeuralNetwork(X_train, y_train, alpha, hidden_layer_sizes, epochs):
#     init_val_division = 1 / 0.1
#     layer_size = [len(X_train[-1])] + hidden_layer_sizes + [1]
#     dims = []
#     for layer, d in enumerate(layer_size):
#         try:
#             temp = (d + 1, layer_size[layer + 1])
#             dims.append(np.array([np.ones(temp) / init_val_division]))
#         except Exception as e:
#             pass
#     W = np.array([dims]).T
#     return W
def fit_NeuralNetwork(X_train, y_train, alpha, hidden_layer_sizes, epochs):
    layer_units = ([len(X_train[-1])] + hidden_layer_sizes + [len(y_train[-1])])
    W = [np.empty((n_fan_in_ + 1, n_fan_out_)) for n_fan_in_,
                                                        n_fan_out_ in zip(layer_units[:-1],
                                                                       layer_units[1:])]
    X_train = np.insert(X_train, 0, 1, axis=1)
    for _ in range(epochs):
        for N, x_n in enumerate(X_train):
            X_n ,S_n = forwardPropagation(x_n, W)
            g_n = backPropagation(X_n, y_train[N], S_n, W)
            W = updateWeights(W, g_n, alpha)
    return W

In [482]:
w = fit_NeuralNetwork([[2]],[[1],[-1]],1,[2,1],1)
print(w)

Wl [[1.        ]
 [0.90154565]]
gl [[-0.00744834]]
der [[0.]]
------------------------------
Wl [[ 4.]
 [ 8.]
 [12.]]
gl [[-0.]]
der [[0.04753486 0.        ]
 [0.         0.00063339]]
------------------------------
[1 2]
[[0. 0.]]
------------------------------
[1.         0.97594321 0.99968325]
[[-0.]]
------------------------------
[1. 1.]
[[-0.00744834]]
------------------------------
[array([[0.44083838, 0.87503983],
       [0.88167675, 1.75007965]]), array([[ 4.],
       [ 8.],
       [12.]]), array([[1.00744834],
       [0.90899399]])]
